In [1]:
import io, os
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
path = 'emails/'

In [3]:
# dataFiles = []
# for root, folder, files in os.walk(path):
#     for file in files:
#         dataFiles.append(root+'/'+file)

In [9]:
file = open('emails/ham/00002.9c4069e25e1ef370c078db7ee85ff9ac')
data = file.readlines()
file.close()

In [10]:
body = False
d = []
for line in data:
    if line == "\n":
        body = True
    if body:
        d.append(line)

In [11]:
d

['\n',
 'Martin A posted:\n',
 'Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the\n',
 ' limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the\n',
 ' Mount Athos monastic community, was ideal for the patriotic sculpture. \n',
 ' \n',
 " As well as Alexander's granite features, 240 ft high and 170 ft wide, a\n",
 ' museum, a restored amphitheatre and car park for admiring crowds are\n',
 'planned\n',
 '---------------------\n',
 'So is this mountain limestone or granite?\n',
 "If it's limestone, it'll weather pretty fast.\n",
 '\n',
 '------------------------ Yahoo! Groups Sponsor ---------------------~-->\n',
 '4 DVDs Free +s&p Join Now\n',
 'http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM\n',
 '---------------------------------------------------------------------~->\n',
 '\n',
 'To unsubscribe from this group, send an email to:\n',
 'forteana-unsubscribe@egroups.com\n',
 '\n',
 ' \n',
 '\n',
 'Your use of Yahoo! Groups is subject t

In [18]:
def read_file(path):
    for root, folder, files in os.walk(path):
        for file in files:
            filename = root+'/'+file
            file = io.open(filename,encoding='latin1')
            body = False
            data = []
            lines = file.readlines()
            for line in lines:
                if line == '\n':
                    body = True
                if body:
                    data.append(line)
            file.close()
            msg = '\n'.join(data)
            yield msg, filename
                    

In [19]:
def df(path, classification):
    rows = []
    index = []
    for message, filename in read_file(path):
        rows.append({'message':message,'class':classification})
        index.append(filename)
        
    return pd.DataFrame(data=rows, index=index)

dataset = pd.DataFrame({'message':[], 'class':[]})
dataset = dataset.append(df('emails/ham', 'ham'))
dataset = dataset.append(df('emails/spam', 'spam'))

In [20]:
dataset.head()

,class,message
emails/ham/00001.7c53336b37003a9286aba55d2945844c,ham,"\n\n Date: Wed, 21 Aug 2002 10:54:46..."
emails/ham/00002.9c4069e25e1ef370c078db7ee85ff9ac,ham,"\n\nMartin A posted:\n\nTassos Papadopoulos, t..."
emails/ham/00003.860e3c3cee1b42ead714c5c874fe25f7,ham,\n\nMan Threatens Explosion In Moscow \n\n\n\n...
emails/ham/00004.864220c5b6930b209cc287c361c99af1,ham,\n\nKlez: The Virus That Won't Die\n\n \n\nAlr...
emails/ham/00005.bf27cdeaf0b8c4647ecd61b1d09da613,ham,"\n\n> in adding cream to spaghetti carbonara,..."


In [55]:
from nltk.tokenize import word_tokenize, sent_tokenize, punkt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix

In [34]:
msg = dataset['message'][0]

In [35]:
import re

In [44]:
# msg = re.findall('[\w]+',msg)
tokens = []
for i in range(len(dataset)):
    msg = re.findall('[\w]+',dataset['message'][i])
    tokens.append(msg)

In [46]:
# tokens[0]

In [47]:
def textProcessing(df):
    tokens = []
    for i in range(len(df)):
#         tokens.append(word_tokenize(df['message'][i]))
        msg = re.findall('[\w]+',df['message'][i])
        tokens.append(msg)

    stopwordsList = stopwords.words("english")
#     stopwordsList.extend([',','.','-','!',':','>','<','\n','@','#','']

    wordsList = []
    for tokenList in tokens:
        words = []
        for word in tokenList:
            if word.lower() not in stopwordsList:
                words.append(word.lower())
        wordsList.append(words)

    wnet = WordNetLemmatizer()
    for i in range(len(wordsList)):
        for j in range(len(wordsList[i])):
            wordsList[i][j] = wnet.lemmatize(wordsList[i][j], pos='v')

    for i in range(len(wordsList)):
        wordsList[i] = ' '.join(wordsList[i])

    return wordsList

In [48]:
wordsList = textProcessing(dataset)

In [50]:
wordsList[1]

'martin post tassos papadopoulos greek sculptor behind plan judge limestone mount kerdylio 70 miles east salonika far mount athos monastic community ideal patriotic sculpture well alexander granite feature 240 ft high 170 ft wide museum restore amphitheatre car park admire crowd plan mountain limestone granite limestone weather pretty fast yahoo group sponsor 4 dvds free p join http us click yahoo com pt6ybb nxieaa mg3haa 7gsolb tm unsubscribe group send email forteana unsubscribe egroups com use yahoo group subject http docs yahoo com info term'

In [51]:
cv = CountVectorizer()

In [52]:
vect = cv.fit_transform(wordsList)

In [53]:
vect = vect.toarray()

In [54]:
y = dataset['class']

In [56]:
encoder = LabelEncoder()

In [57]:
y = encoder.fit_transform(y)

In [58]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [63]:
x_train, x_test, y_train, y_test = train_test_split(vect, y, test_size=0.25)

In [64]:
x_train.shape

(2250, 58388)

In [65]:
x_test.shape

(750, 58388)

In [66]:
y_train.shape

(2250,)

In [67]:
y_test.shape

(750,)

In [68]:
reg = LogisticRegression()
reg.fit(x_train, y_train)

C:\Users\asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [69]:
y_pred = reg.predict(x_test)

In [70]:
accuracy_score(y_test, y_pred)

0.9866666666666667

In [72]:
nb = GaussianNB()

In [73]:
nb.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [74]:
y_pred_2 = nb.predict(x_test)

In [75]:
accuracy_score(y_test, y_pred_2)

0.9666666666666667